In [1]:
# 📘 Updated Cell-by-Cell Notebook: DANN Multi-Task Model with Domain Output

# 📌 Cell 1: Imports and Constants
import os
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbCallback
from tensorflow.keras import layers, models, callbacks
from collections import Counter
import numpy as np

# Constants
NUM_CROPS = 9
NUM_DISEASES = 33
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE


2025-03-24 18:51:00.179461: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 18:51:00.194308: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742822460.210613    8378 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742822460.216080    8378 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 18:51:00.232734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# 📌 Cell 2: Initialize W&B with config
wandb.init(
    project="plantvillage_dann_model",
    name="DANN_EfficientNetV2B0_RUN_2",
    config={
        "model": "EfficientNetV2B0_DANN",
        "optimizer": "Adam",
        "learning_rate": 1e-4,
        "batch_size": BATCH_SIZE,
        "loss_weights": {"crop_output": 1.0, "disease_output": 2.0, "domain_output": 0.5},
        "epochs": 30,
        "augmentation": "None",
        "include_top": False,
        "input_shape": (224, 224, 3)
    }
)



wandb: Currently logged in as: gsrivatsa1398 (gsrivatsa1398-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
# 📌 Cell 3: TFRecord Paths
split_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Dataset/PlantVillage_Structured/AUG_TFRecord/split"
train_tfrecord = os.path.join(split_path, "train.tfrecord")
val_tfrecord = os.path.join(split_path, "val.tfrecord")
test_tfrecord = os.path.join(split_path, "test.tfrecord")



In [4]:
# 📌 Cell 4: TFRecord Parsing Functions
feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "crop": tf.io.FixedLenFeature([], tf.int64),
    "disease": tf.io.FixedLenFeature([], tf.int64)
}

def parse_tfrecord(example_proto):
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed["image"], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0

    crop = tf.one_hot(parsed["crop"], depth=NUM_CROPS)
    disease = tf.one_hot(parsed["disease"], depth=NUM_DISEASES)
    domain = crop  # Domain label = crop label

    return image, {"crop_output": crop, "disease_output": disease, "domain_output": domain}



In [5]:
# 📌 Cell 5: Load Dataset Function
def load_dataset(path):
    dataset = tf.data.TFRecordDataset(path)
    dataset = dataset.map(parse_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return dataset

train_dataset = load_dataset(train_tfrecord)
val_dataset = load_dataset(val_tfrecord)
test_dataset = load_dataset(test_tfrecord)



I0000 00:00:1742822469.651751    8378 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [6]:
# 📌 Cell 6: Compute Class Weights
def compute_class_weights(dataset):
    crop_counter = Counter()
    disease_counter = Counter()
    for batch in dataset:
        _, labels = batch
        crops = tf.argmax(labels["crop_output"], axis=1).numpy()
        diseases = tf.argmax(labels["disease_output"], axis=1).numpy()
        crop_counter.update(crops.tolist())
        disease_counter.update(diseases.tolist())

    total_crops = sum(crop_counter.values())
    total_diseases = sum(disease_counter.values())

    crop_weights = {i: total_crops / (len(crop_counter) * crop_counter[i]) for i in crop_counter}
    disease_weights = {i: total_diseases / (len(disease_counter) * disease_counter[i]) for i in disease_counter}

    return crop_weights, disease_weights

crop_class_weights, disease_class_weights = compute_class_weights(train_dataset)
crop_weights_array = np.array([crop_class_weights.get(i, 1.0) for i in range(NUM_CROPS)], dtype=np.float32)
disease_weights_array = np.array([disease_class_weights.get(i, 1.0) for i in range(NUM_DISEASES)], dtype=np.float32)



2025-03-24 18:51:10.072799: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:370] TFRecordDataset `buffer_size` is unspecified, default to 262144
2025-03-24 18:51:13.693248: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
# 📌 Cell 7: Custom Weighted Loss Function
def weighted_categorical_crossentropy(weights_array):
    weights_tensor = tf.constant(weights_array, dtype=tf.float32)
    def loss_fn(y_true, y_pred):
        base_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        true_labels = tf.argmax(y_true, axis=-1)
        sample_weights = tf.gather(weights_tensor, true_labels)
        weighted_loss = base_loss * sample_weights
        return tf.reduce_mean(weighted_loss)
    return loss_fn



In [8]:
# 📌 Cell 8: Build DANN Multi-Task Model
input_image = layers.Input(shape=(224, 224, 3))
base_model = tf.keras.applications.EfficientNetV2B0(include_top=False, weights="imagenet", pooling="avg")
features = base_model(input_image)

shared = layers.Dense(256, activation=None, kernel_regularizer=tf.keras.regularizers.l2(0.001))(features)
shared = layers.BatchNormalization()(shared)
shared = layers.Activation("relu")(shared)
shared = layers.Dropout(0.3)(shared)

crop_output = layers.Dense(NUM_CROPS, activation="softmax", name="crop_output")(shared)
disease_output = layers.Dense(NUM_DISEASES, activation="softmax", name="disease_output")(shared)

# Domain Discriminator
domain_dense = layers.Dense(128, activation="relu")(shared)
domain_output = layers.Dense(NUM_CROPS, activation="softmax", name="domain_output")(domain_dense)

model = models.Model(inputs=input_image, outputs=[crop_output, disease_output, domain_output])



In [9]:
# 📌 Cell 9: Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss={
        "crop_output": weighted_categorical_crossentropy(crop_weights_array),
        "disease_output": weighted_categorical_crossentropy(disease_weights_array),
        "domain_output": "categorical_crossentropy"
    },
    loss_weights={"crop_output": 1.0, "disease_output": 2.0, "domain_output": 0.5},
    metrics={"crop_output": "accuracy", "disease_output": "accuracy", "domain_output": "accuracy"}
)



In [10]:
# 📌 Cell 10: Callbacks
model_save_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/DANN/dann_model.keras"
cb = [
    callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
    callbacks.ModelCheckpoint(model_save_path, save_best_only=True, monitor="val_disease_output_accuracy", mode="max"),
    WandbCallback(log_model=False, save_model=False,save_graph=False)
]


wandb: WARNING WandbCallback is deprecated and will be removed in a future release. Please use the WandbMetricsLogger, WandbModelCheckpoint, and WandbEvalCallback callbacks instead. See https://docs.wandb.ai/guides/integrations/keras for more information.


In [11]:
# 📌 Cell 11: Train Model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=cb
)


Epoch 1/30


I0000 00:00:1742822502.950493    8536 service.cc:148] XLA service 0x7fc40c015d50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1742822502.950560    8536 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-03-24 18:51:43.945595: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1742822507.954686    8536 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-03-24 18:51:50.568924: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22741_0', 428 bytes spill stores, 684 bytes spill loads

2025-03-24 18:51:50.677955: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot

    496/Unknown 147s 145ms/step - crop_output_accuracy: 0.5627 - crop_output_loss: 1.1771 - disease_output_accuracy: 0.4900 - disease_output_loss: 2.0066 - domain_output_accuracy: 0.5460 - domain_output_loss: 1.3839 - loss: 6.3078

2025-03-24 18:53:48.291519: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22741', 8 bytes spill stores, 8 bytes spill loads

2025-03-24 18:53:48.383665: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22741', 252 bytes spill stores, 252 bytes spill loads

2025-03-24 18:53:48.618568: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22741_0', 8 bytes spill stores, 8 bytes spill loads

2025-03-24 18:53:48.836813: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23774', 96 bytes spill stores, 96 bytes spill loads

2025-03-24 18:53:48.912503: I external/local_xla/xla/str

    497/Unknown 188s 229ms/step - crop_output_accuracy: 0.5631 - crop_output_loss: 1.1759 - disease_output_accuracy: 0.4905 - disease_output_loss: 2.0047 - domain_output_accuracy: 0.5465 - domain_output_loss: 1.3827 - loss: 6.3022

...

497/497 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - crop_output_accuracy: 0.9999 - crop_output_loss: 8.8235e-04 - disease_output_accuracy: 0.9996 - disease_output_loss: 0.0016 - domain_output_accuracy: 1.0000 - domain_output_loss: 4.3722e-04 - loss: 0.0947      

2025-03-24 19:28:52.484723: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13371830520981379079
2025-03-24 19:28:52.484793: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 16911993988210025755
2025-03-24 19:28:52.484806: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 5446009930160431017
2025-03-24 19:28:52.484816: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1674703350760878374
2025-03-24 19:28:59.916484: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13371830520981379079
2025-03-24 19:28:59.916614: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1674703350760878374


497/497 ━━━━━━━━━━━━━━━━━━━━ 80s 161ms/step - crop_output_accuracy: 0.9999 - crop_output_loss: 8.8224e-04 - disease_output_accuracy: 0.9996 - disease_output_loss: 0.0016 - domain_output_accuracy: 1.0000 - domain_output_loss: 4.3704e-04 - loss: 0.0947 - val_crop_output_accuracy: 0.9997 - val_crop_output_loss: 0.0023 - val_disease_output_accuracy: 0.9961 - val_disease_output_loss: 0.0211 - val_domain_output_accuracy: 0.9996 - val_domain_output_loss: 0.0021 - val_loss: 0.1347 - learning_rate: 6.2500e-06
Epoch 28/30
497/497 ━━━━━━━━━━━━━━━━━━━━ 78s 157ms/step - crop_output_accuracy: 0.9999 - crop_output_loss: 8.9763e-04 - disease_output_accuracy: 0.9996 - disease_output_loss: 0.0017 - domain_output_accuracy: 0.9997 - domain_output_loss: 5.3969e-04 - loss: 0.0933 - val_crop_output_accuracy: 0.9998 - val_crop_output_loss: 0.0023 - val_disease_output_accuracy: 0.9959 - val_disease_output_loss: 0.0237 - val_domain_output_accuracy: 0.9996 - val_domain_output_loss: 0.0020 - val_loss: 0.1383 - le

2025-03-24 19:31:40.286233: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 13371830520981379079
2025-03-24 19:31:40.286299: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1674703350760878374


497/497 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - crop_output_accuracy: 0.9999 - crop_output_loss: 7.8949e-04 - disease_output_accuracy: 0.9999 - disease_output_loss: 0.0012 - domain_output_accuracy: 1.0000 - domain_output_loss: 3.5014e-04 - loss: 0.0904 - val_crop_output_accuracy: 0.9997 - val_crop_output_loss: 0.0023 - val_disease_output_accuracy: 0.9964 - val_disease_output_loss: 0.0229 - val_domain_output_accuracy: 0.9996 - val_domain_output_loss: 0.0020 - val_loss: 0.1348 - learning_rate: 6.2500e-06
Epoch 30/30
497/497 ━━━━━━━━━━━━━━━━━━━━ 79s 158ms/step - crop_output_accuracy: 0.9996 - crop_output_loss: 0.0014 - disease_output_accuracy: 0.9997 - disease_output_loss: 0.0017 - domain_output_accuracy: 0.9996 - domain_output_loss: 8.3030e-04 - loss: 0.0905 - val_crop_output_accuracy: 0.9997 - val_crop_output_loss: 0.0024 - val_disease_output_accuracy: 0.9965 - val_disease_output_loss: 0.0195 - val_domain_output_accuracy: 0.9996 - val_domain_output_loss: 0.0020 - val_loss: 0.1263 - learni

In [12]:
# 📌 Cell 12: Evaluate Model
test_metrics = model.evaluate(test_dataset)
print("\n✅ Final Test Evaluation:", test_metrics)

     70/Unknown 5s 63ms/step - crop_output_accuracy: 0.9996 - crop_output_loss: 9.7919e-04 - disease_output_accuracy: 0.9959 - disease_output_loss: 0.0233 - domain_output_accuracy: 0.9996 - domain_output_loss: 0.0011 - loss: 0.1321     

2025-03-24 19:33:08.211703: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2447', 4 bytes spill stores, 4 bytes spill loads

2025-03-24 19:33:08.907507: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2447_0', 256 bytes spill stores, 256 bytes spill loads

2025-03-24 19:33:08.952723: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2447', 252 bytes spill stores, 252 bytes spill loads



71/71 ━━━━━━━━━━━━━━━━━━━━ 15s 207ms/step - crop_output_accuracy: 0.9996 - crop_output_loss: 9.8160e-04 - disease_output_accuracy: 0.9959 - disease_output_loss: 0.0234 - domain_output_accuracy: 0.9996 - domain_output_loss: 0.0011 - loss: 0.1322

✅ Final Test Evaluation: [0.13785307109355927, 0.001065970049239695, 0.0261083897203207, 0.0012442773440852761, 0.9995595812797546, 0.994494616985321, 0.9995595812797546]
